# Working RAG with Generative AI 🤖

Data Science utiliza un Modelo LLMs [Cohere Command R+] para analizar los documentos indexados en OpenSearch y realizar acciones basadas en el contenido de los documentos. El modelo es personalizado en Data Science y desplegado para poder ser consumido por las aplicaciones.

[![Notebook Examples](https://img.shields.io/badge/docs-notebook--examples-blue)](https://github.com/oracle-samples/oci-data-science-ai-samples/tree/master/notebook_examples)
[![Conda Environments](https://img.shields.io/badge/docs-conda--environments-blue)](https://docs.oracle.com/en-us/iaas/data-science/using/conda_understand_environments.htm)
[![Source Code](https://img.shields.io/badge/source-accelerated--datascience-blue)](https://github.com/oracle/accelerated-data-science)

##### [Step-01] Libraries

<details>
<summary><font size="2">Install Pre-Requirements</font></summary>
<font size="1">

```Install Libraries
(base) bash-4.2$ odsc conda create -f environment.yaml -n langchain_env -v 1.5
```
    
</font>    
</details>

In [1]:
import oci
import ads
import os

import requests
import json

from pathlib import Path

#https://api.python.langchain.com/en/latest/embeddings/langchain_community.embeddings.oci_generative_ai.OCIGenAIEmbeddings.html
from langchain_community.embeddings.oci_generative_ai import OCIAuthType
from langchain_community.embeddings.oci_generative_ai import OCIGenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.chat_models.oci_generative_ai import ChatOCIGenAI
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

##### [Step-02] Parameters

In [2]:
# oci: Generative AI
compartment_id   = os.environ['NB_SESSION_COMPARTMENT_OCID']
service_endpoint = "https://inference.generativeai.us-chicago-1.oci.oraclecloud.com"
genai_embeddings = 'cohere.embed-multilingual-v3.0'
genai_inference  = "ocid1.generativeaimodel.oc1.us-chicago-1.***"
auth_type        = "RESOURCE_PRINCIPAL"

# oci: OpenSearch
apiEndpoint = "http://***.***.***.***:9200"
username    = "opensearch"
password    = "************"
searchIndex = "oci_documents"

##### [Step-01] Opensearch
Function to retrieve documents from OpenSearch based on the provided obj_url

In [3]:
def get_documents_opensearch(obj_url):
    content = ""    
    # Construct the search URL for the OpenSearch endpoint
    queryurl = f"{apiEndpoint}/{searchIndex}/_search"
    # Set the authentication credentials for the OpenSearch request
    auth = (username, password)
    # Set the headers for the OpenSearch request
    headers = {"Content-Type": "application/json"}
    # Construct the OpenSearch query
    query = {
        "query": {
            "bool": {
                "must": [
                    { "term": { "obj_url.keyword": obj_url } }
                ]
            }
        },
        "_source": ["obj_content"]  # Fetch only the obj_content field
    }
    
    try:
        # Send a GET request to the OpenSearch endpoint with the constructed query
        resp = requests.get(queryurl, auth=auth, headers=headers, data=json.dumps(query))
        # Raise an exception if the request returned an unsuccessful status code
        resp.raise_for_status()
        # Parse the JSON response from the request
        response = resp.json()
        
        # Iterate over the search results in the response
        for hit in response['hits']['hits']:
            content += hit['_source']['obj_content'] + "\n\n"
          
        return content        
    except requests.exceptions.RequestException as e:
        print(f"[get_documents_opensearch] {str(e)}.")
        raise

##### [Step-02] Embedding
Function to retrieve embeddings from OCI Generative AI service

In [4]:
def get_embeddings():
    try:
        # Initialize the OCIGenAIEmbeddings object with the specified parameters
        embeddings = OCIGenAIEmbeddings(
            model_id         = genai_embeddings, # Specify the model ID for embeddings
            service_endpoint = service_endpoint,
            compartment_id   = compartment_id,
            auth_type        = auth_type # Authentication type
        )
        
        return embeddings
    except requests.exceptions.RequestException as e:
        print(f"[get_embeddings_model] {str(e)}.")
        raise

 ##### [Step-03] Retriever
 Function to generate content embeddings and create a retriever for searching the content


In [5]:
def get_content(text, embeddings):    
    try:
        # Generate embeddings for the content
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size         = 1000,  # Size of each text chunk
            chunk_overlap      = 50,    # Overlap between chunks
            length_function    = len,   # Function to measure the length of text
            is_separator_regex = False  # Whether the separator is a regex
        )
        
        # Split the text into chunks
        chunks = text_splitter.split_text(text)

        # Create the FAISS vector store with the embeddings
        VectorStore = FAISS.from_texts(chunks, embeddings)

        # Convert the vector store to a retriever
        retriever = VectorStore.as_retriever(search_kwargs={"k": 1000}) # Number of top results to retrieve
        
        return retriever
    except requests.exceptions.RequestException as e:
        print(f"[get_content] {str(e)}.")
        raise

##### [Step-04] LLM 

<details>
<summary><font size="2">ChatOCIGenAI</font></summary>
<font size="1">
        
```text
> model_id: Specify the model ID for the chat model "cohere.command-r-plus".
> service_endpoint: The service endpoint for OCI Generative AI
> compartment_id: OCI compartment ID
> provider: The provider of the model
> is_stream: Whether to use streaming for responses
> auth_type: Authentication type
> max_tokens: Maximum number of tokens in the response
> temperature: Sampling temperature
> top_p: Nucleus sampling parameter
> top_k: Top-K sampling parameter
> frequency_penalty: Frequency penalty for token repetition
```
        
</font>
</details>

In [6]:
# Define the object URL for the document in OCI Object Storage
obj_url   = "https://objectstorage.us-chicago-1.oraclecloud.com/n/idi1o0a010nx/b/DLK1LAGDEV/o/example.xlsx"

# Retrieve the document text from OpenSearch based on the object URL
text = get_documents_opensearch(obj_url)

# Get the embeddings using the specified model and configuration
embeddings = get_embeddings()

# Generate content embeddings and create a retriever for searching the content
retriever   = get_content(text, embeddings)

# Initialize the ChatOCIGenAI object with the specified parameters
chat = ChatOCIGenAI(
    model_id         = "cohere.command-r-plus",
    service_endpoint = service_endpoint,
    compartment_id   = compartment_id,
    provider         = "cohere",
    is_stream        = True,
    auth_type        = auth_type,
    model_kwargs     = {
        "max_tokens": 512,
        "temperature": 0.6,
        "top_p": 0.9,
        "top_k": 20,
        "frequency_penalty": 1
    }
)

# Define a prompt template for the chat model
prompt_template = PromptTemplate.from_template("{query}, basándose únicamente en el siguiente contexto: {context}")

# Create a processing chain with the query, retriever, prompt template, chat model, and output parser
chain = (
    {"query": RunnablePassthrough(), "context": retriever}  # Pass the query and retriever context
    | prompt_template                                       # Apply the prompt template
    | chat                                                  # Invoke the chat model
    | StrOutputParser()                                     # Parse the output as a string
)

# Invoke the chain with the query (max: 250 character)
chain.invoke("Generar un resumen")

'Aquí está un resumen del contexto proporcionado:\n\nEl documento parece ser una lista de artículos y repuestos con sus respectivas descripciones, cantidades y valores ofrecidos. Hay un total de 40 artículos en la lista, que van desde retenes de aceite y pernos hasta filtros de aire y aceite, fusibles y varios tipos de cintas aisladoras. Cada artículo tiene un número de orden, una descripción detallada, la categoría a la que pertenece y el valor ofrecido. El documento también incluye información sobre el subtotal, el IVA y el total de la transacción, aunque todos estos valores son 0 en este caso particular.'